# Chap08 映画レビューの分類
* データの獲得
* データの前処理
* モデルの訓練

In [1]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2020-05-01 08:41:34--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.2’

aclImdb_v1.tar.gz.2 100%[===================>]  80.23M  17.2MB/s    in 10s     

2020-05-01 08:41:44 (8.01 MB/s) - ‘aclImdb_v1.tar.gz.2’ saved [84125825/84125825]



In [0]:
!tar -zxf aclImdb_v1.tar.gz

In [3]:
!pip install PyPrind

In [4]:
import pyprind
import pandas as pd
import os

# change the `basepath` to the directory of the
# unzipped movie dataset

#basepath = '/Users/Sebastian/Desktop/aclImdb/'
basepath = './aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:23


In [0]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [0]:
df.to_csv('./movie_data.csv', index=False)

In [7]:
import pandas as pd

df = pd.read_csv('./movie_data.csv')
df.head(3)

,review,sentiment
0,There have been very few great comedy films in...,1
1,"Oh My God, this is so idiotic. Completely poin...",0
2,"Jack Frost, no kids it's not the warm hearted ...",0


In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [10]:
next(stream_docs(path='./movie_data.csv'))

('"There have been very few great comedy films in the history of Hindi Cinema. Andaz Apna Apna happens to be one of them. The film is based on a very simple story of two poor young men (Aamir and Salman) who dream of becoming rich by marrying a millionaire\'s daughter (Raveena). Aamir and Salman Khan try their best to outwit each other and woo Raveena. The plot thickens when Paresh Rawal & Co. plan to take over all the wealth. The movie is well paced and very funny. Rarely does one come across a Hindi comedy which is both funny and intelligent. This is one of the few films with Aamir and Salman together (probably the only film!). Unfortunately it did not succeed at the box-office, and we might never see a film of this calibre again. Aamir Khan is brilliant in the film and has proved his versatility as an actor in this film. Salman Khan gives a very good performance as a dim-wit. Raveena plays a convincing role as a confused rich girl, and Karishma who is Raveena\'s assistant/friend is 

In [0]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [0]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='./movie_data.csv')

In [13]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:27


In [14]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


In [0]:
clf = clf.partial_fit(X_test, y_test)

# 学習済みの scikit-learn 推定器をシリアライズする

In [0]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

colab だと \_\_file\_\_ が定義されていないらしい  
代わりに dest = os.path.join('movieclassifier', 'pkl_objects') を使う

In [0]:
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

stop = pickle.load(open(
                os.path.join(dest, 
                'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

In [0]:
import os
os.chdir('movieclassifier')

In [19]:
!pwd

/content/movieclassifier


In [0]:
import pickle
import re
import os

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

In [21]:
import numpy as np
label = {0:'negative', 1:'positive'}

example = ['I love this movie']
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf.predict(X)[0]], clf.predict_proba(X).max()*100))

Prediction: positive
Probability: 82.01%


# データストレージとして SQLite データベースをセットアップする

In [0]:
import sqlite3
import os

if os.path.exists('reviews.sqlite'):
    os.remove('reviews.sqlite')

conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example2, 0))

conn.commit()
conn.close()

In [0]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()

c.execute("SELECT * FROM review_db WHERE date BETWEEN '2015-01-01 10:10:10' AND DATETIME('now')")
results = c.fetchall()

conn.close()

In [24]:
print(results)

[('I love this movie', 1, '2020-05-01 08:43:57'), ('I disliked this movie', 0, '2020-05-01 08:43:57')]
